In [1]:
import pandas as pd
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 
cid ="5cf7fd8a74434b4aa2243cd27dc0e490" 
secret = "033870832e60454b8c23e5e7a56b4280" 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0,1000,50): 
    track_results = sp.search(q='year:2022', type='track', limit=50,offset=i) 
    for i, t in enumerate(track_results['tracks']['items']):
           artist_name.append(t['artists'][0]['name']) 
           track_name.append(t['name']) 
           track_id.append(t['id']) 
           popularity.append(t['popularity'])


df_tracks = pd.DataFrame({'popularity':popularity,'artist_name':artist_name,'track_name':track_name,'track_id':track_id})
# df_tracks = df_tracks.sort_values('popularity', ascending=False)
print(df_tracks.shape)
df_tracks.head()

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

(1000, 4)


32

In [2]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [3]:
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [4]:
df_tracks.shape

(955, 4)

In [5]:
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

Number of tracks where no audio features were available: 5


In [6]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (950, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.911,0.712,1,-5.105,0,0.0817,0.09010,0.000027,0.0933,0.425,92.005,audio_features,6Sq7ltF9Qa7SNFBsV5Cogx,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4
1,0.521,0.683,5,-6.240,0,0.5010,0.15900,0.000000,0.1870,0.737,139.439,audio_features,4eHkG6yWQ4pD0zqVkV0R5a,spotify:track:4eHkG6yWQ4pD0zqVkV0R5a,https://api.spotify.com/v1/tracks/4eHkG6yWQ4pD...,https://api.spotify.com/v1/audio-analysis/4eHk...,161808,3
2,0.824,0.697,11,-4.869,0,0.0473,0.15800,0.000041,0.0770,0.757,99.988,audio_features,7fYRg3CEbk6rNCuzNzMT06,spotify:track:7fYRg3CEbk6rNCuzNzMT06,https://api.spotify.com/v1/tracks/7fYRg3CEbk6r...,https://api.spotify.com/v1/audio-analysis/7fYR...,214460,4
3,0.658,0.935,2,-6.030,0,0.0679,0.00948,0.670000,0.0536,0.919,124.006,audio_features,6m5Pu0B2YVPR8BkrbKvHuE,spotify:track:6m5Pu0B2YVPR8BkrbKvHuE,https://api.spotify.com/v1/tracks/6m5Pu0B2YVPR...,https://api.spotify.com/v1/audio-analysis/6m5P...,173161,4
4,0.686,0.507,1,-7.097,1,0.0357,0.62600,0.000075,0.3830,0.690,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4


In [7]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      950 non-null    float64
 1   energy            950 non-null    float64
 2   key               950 non-null    int64  
 3   loudness          950 non-null    float64
 4   mode              950 non-null    int64  
 5   speechiness       950 non-null    float64
 6   acousticness      950 non-null    float64
 7   instrumentalness  950 non-null    float64
 8   liveness          950 non-null    float64
 9   valence           950 non-null    float64
 10  tempo             950 non-null    float64
 11  type              950 non-null    object 
 12  id                950 non-null    object 
 13  uri               950 non-null    object 
 14  track_href        950 non-null    object 
 15  analysis_url      950 non-null    object 
 16  duration_ms       950 non-null    int64  
 1

In [8]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(950, 14)

In [9]:
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (950, 14)


,popularity,artist_name,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,99,Bad Bunny,Me Porto Bonito,6Sq7ltF9Qa7SNFBsV5Cogx,0.911,0.712,1,-5.105,0,0.0817,0.09010,0.000027,0.0933,0.425,92.005,178567,4
1,20,Ish Kevin,My Year (2022),4eHkG6yWQ4pD0zqVkV0R5a,0.521,0.683,5,-6.240,0,0.5010,0.15900,0.000000,0.1870,0.737,139.439,161808,3
2,84,Calvin Harris,Potion (with Dua Lipa & Young Thug),7fYRg3CEbk6rNCuzNzMT06,0.824,0.697,11,-4.869,0,0.0473,0.15800,0.000041,0.0770,0.757,99.988,214460,4
3,0,Avicii,Street Dancer - Sgt Slick's Discotizer 2022 Remix,6m5Pu0B2YVPR8BkrbKvHuE,0.658,0.935,2,-6.030,0,0.0679,0.00948,0.670000,0.0536,0.919,124.006,173161,4
4,91,Steve Lacy,Bad Habit,5CM4UuQ9Gnd6K2YyKGPMoK,0.686,0.507,1,-7.097,1,0.0357,0.62600,0.000075,0.3830,0.690,168.955,232115,4


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 0 to 949
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   popularity        950 non-null    int64  
 1   artist_name       950 non-null    object 
 2   track_name        950 non-null    object 
 3   track_id          950 non-null    object 
 4   danceability      950 non-null    float64
 5   energy            950 non-null    float64
 6   key               950 non-null    int64  
 7   loudness          950 non-null    float64
 8   mode              950 non-null    int64  
 9   speechiness       950 non-null    float64
 10  acousticness      950 non-null    float64
 11  instrumentalness  950 non-null    float64
 12  liveness          950 non-null    float64
 13  valence           950 non-null    float64
 14  tempo             950 non-null    float64
 15  duration_ms       950 non-null    int64  
 16  time_signature    950 non-null    int64  
dt

In [11]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,popularity,artist_name,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [12]:
df.to_csv('SpotifyAudioFeatures08082018.csv')